In [ ]:
addprocs([
    ("134.173.61.242", 4), 
    ("134.173.60.166", 4)
])

In [ ]:
# Generate a mutable version of JLOptions
@everywhere field_names = fieldnames(Base.JLOptions)
@everywhere field_types = Base.JLOptions.types
@everywhere num_fields = length(field_names)

@everywhere fields = Array{Expr}(num_fields)
@everywhere for i = 1:num_fields
    fields[i] = :($(field_names[i])::$(field_types[i]))
end

@everywhere @eval begin
    type JLOptionsMutable
        $(fields...)
    end
end

@everywhere function use_compilecache(state::Bool)
    value = Base.convert(Int, state)

    # Load the C global into a mutable Julia type
    jl_options = cglobal(:jl_options, JLOptionsMutable)
    opts = unsafe_load(jl_options)

    # Avoid modifying the global when the value hasn't changed
    if opts.use_compilecache != value
        opts.use_compilecache = value
        unsafe_store!(jl_options, opts)
    end

    nothing
end
@everywhere use_compilecache(false)

In [ ]:
import DifferentialEquations
import ParameterizedFunctions
@everywhere using DifferentialEquations
@everywhere using ParameterizedFunctions

In [ ]:
@everywhere g = @ode_def_bare Sipz2 begin    
    dp = sin(t)
    dk = kv
    dkv = sign(p - k)/m
end m=>1

In [ ]:
@everywhere u0 = [0.0;0.0;0.0;0.0;0.0;0.0]
@everywhere g.m = 2500
@everywhere tspan = (0.0,10000000.0)
@everywhere prob = ODEProblem(g,u0,tspan)

@everywhere sol = solve(prob, Trapezoid(), dt=0.01)

In [ ]:
@time t = linspace(0,1000,100000)
@time r = hcat(map(sol, t)...)
p, k = r[1,:], r[2,:]
plot(t, p)
plot!(t, k)